In [1]:
import json
import os

import shutil
import base64
import requests
import pandas as pd 

from dotenv import load_dotenv
import google.generativeai as genai

In [3]:
# fdata = []
# data = []
# with open("dataset/chitron-captions-fresh.json", "r+") as f:
#     data = json.load(f)

In [2]:

load_dotenv()

OPENAI_API = os.getenv("INTAAS_OPENAI_API")
GEMINI_API = os.getenv("GOOGLE_API")

In [3]:
# Import the Images module from pillow 
from PIL import Image 
  

def encode_image(image_path):
        
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')    

In [4]:
def prompt(captions):
    return """
    You are an expert Bengali question, answer generating assistant. You always generate the question and answer from \
    the given image and the captions to the image. you always have to abide by the conditions that are mentioned below: \n \n  
    CONDITION 1:  The questions are always image aligned and caption aligned and informative \n
    CONDITION 2:  Try to generate answer of one or two words. And the answer must never contain more than three words \n
    CONDITION 3:  Generate the question answer pair in Bengali language \n
        \n \n \
    Here is the caption:\n \n {},\
        \n \nBased on the above captions and below given image, generate ONE Question,Answer pair 
        based on this picture in Bengali language. generate the question answer pair like the following format:
    
    Q# "generated question", A# "generated answer"

    """.format(captions)

In [5]:
def give_q_n_a(image_name, captions):
    # Path to your image
    # image_path = f"dataset/chitron/{image_name}"
    image_path = f"dataset/bnature/{image_name}"

    # Encode the image
    base64_image = encode_image(image_path)

    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {OPENAI_API}"
    }
    payload = {
        # "model": "gpt-4-vision-preview", "gpt-4-turbo"
        "model": "gpt-4-turbo",
        # "response_format": {"type": "json_object"},
        "messages": [
            {
                "role": "user",
                "content": [
                    # {"type": "text", "text": "Caption the image explaining the contents in it. Use Bengali language to caption the image."},
                    {"type": "text", "text": prompt(captions)},

                    {"type": "image_url", "image_url": {"url": 
    f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": 100
    }   


    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
    
    return response.json()#['choices'][0]['message']['content']


## Chitron

In [4]:
# df = pd.json_normalize(data)
# df.loc[:,'Generated'] = None

df = pd.read_csv("dataset/chitron_concat.csv")

In [5]:
# df.to_csv("dummy.csv", index=False)
df.head()

,Unnamed: 0,filename,caption,Generated,longest_caption
0,0,1.png,['তিন জন মেয়ে মানুষ আছে। এক জন দাড়িয়ে আছে আর দ...,"Q# ""ছবিতে মহিলাটির হাতে কি আছে?"", A# ""বেত""",'একটি হলুদ জামা পায়জামা পরা মহিলা দাড়িয়ে হাতে...
1,1,2.png,['জলাশয় এ একজন ছেলে মানুষ ও একটি বাচ্চা মানুষ ...,"Q# ""নৌকায় দাঁড়ানো পুরুষটি কি পরে আছেন?"", A# ""স...",'২ জন মানুষ যার একজন পুরুষ একটি নৌকায় দাড়িয়ে ...
2,2,3.png,"['অনেক মেয়ে মানুষ বসে আছে।', 'একটি নীল জামা পর...","Q# ""ল্যাপটপ ধারণকারী মহিলার জামার রঙ কি?"", A# ...",'একটি নীল জামা পরা মহিলা একটি নীল ল্যাপটপ এর ...
3,3,4.png,"['অনেক মানুষ একসাথে বসে কাজ করছে।', '২ টি ছোট...","Q# ""ছবিতে বসে থাকা ছোট ছেলেটি কী ধরনের পোশাক প...",'২ টি ছোট ছেলে একজন শার্ট প্যান্ট দাড়িয়ে চে...
4,4,5.png,"['ছয় জন মানুষ দাড়িয়ে আছে।', '৬ জন মানুষ এলোমেল...","Q# ""এই ছবিতে কতজন পুরুষ লুঙ্গী পরে আছেন?""\nA# ...","'৬ জন মানুষ এলোমেলো দাড়িয়ে আছে, তাদের মাঝে ২ ..."


In [6]:
# validation = df[mask]
df.isna().sum()

Unnamed: 0         0
filename           0
caption            0
Generated          9
longest_caption    0
dtype: int64

In [58]:
import time

sliced_df = df[8660:8669]

In [59]:
# total_input_tokens = 6160179 #4842322 #4266080 #3782210 #3677753 #3562644 #2848053 #2288098 #2176137 #405867 # 293207
# total_output_tokens = 934780 #785276 #718417 #661913 #649949 #636898 #556623 #493624 # 482592 # 1537880 # 293207
# total_tokens = 2520663 #2397670 #1393072 # 325355
#  36.23913 # 3.8965100000000006 $  #45.17922
val = 0

In [60]:
sliced_df.isna().sum()

Unnamed: 0         0
filename           0
caption            0
Generated          9
longest_caption    0
dtype: int64

In [61]:
for idx, rows in sliced_df.iterrows():
    res = (give_q_n_a(rows['filename'], rows['longest_caption']))

    total_input_tokens = total_input_tokens + res['usage']['prompt_tokens']
    total_output_tokens = total_output_tokens + res['usage']['completion_tokens']
    total_tokens = total_tokens + res['usage']['total_tokens']
    
    df.loc[df['filename'] == str(rows['filename']), 'Generated'] = str(res['choices'][0]['message']['content'])
    
    print("done: idx: ",idx," filename: ", str(rows['filename']), "input_token: ", res['usage']['prompt_tokens'], "output_token: ", res['usage']['completion_tokens'])
    val = val+1
    time.sleep(.6)
    # break

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/chitron/10000.png'

In [14]:
# 4016 - 3500
# 4263
# res

In [28]:
sliced_df.head(10)

,Unnamed: 0,filename,caption,Generated,longest_caption
8237,8237,8732.png,['একজন লোক কাঁধে ব্যাগ নিয়ে ক্যামেরা হাতে ছবি ...,Q# ছবিতে ব্যক্তিটি কি করছে? A# ছবি তুলছে।,একজন লোক কাঁধে ব্যাগ নিয়ে ক্যামেরা হাতে ছবি তু...
8238,8238,8733.png,['জানালা দিয়ে দেখা যাচ্ছে একজন মহিলা সেলাই মেশ...,NaN,জানালা দিয়ে দেখা যাচ্ছে একজন মহিলা সেলাই মেশিন...
8239,8239,8734.png,['কুয়াশাছন্ন দিনে একটা ট্রেন লাইনের পাশে একজন ...,NaN,কুয়াশাছন্ন দিনে একটা ট্রেন লাইনের পাশে একজন লো...
8240,8240,8735.png,['একটা বাচ্চা ছেলে একটা টায়ার হাতে নিয়ে বসে আছ...,NaN,একটা বাচ্চা ছেলে একটা টায়ার হাতে নিয়ে বসে আছে।
8241,8241,8736.png,['একটা ফলের দোকানে একজন লোক হাঁসি মুখে বসে আছে...,NaN,একটা ফলের দোকানে একজন লোক হাঁসি মুখে বসে আছে।
8242,8242,8737.png,"['একজন মহিলা বোতল দিয়ে পানি খাচ্ছে।', 'একজন না...",NaN,একজন মহিলা বোতল দিয়ে পানি খাচ্ছে।
8243,8243,8738.png,['দুইজন লোক দুইটা ঘোড়ার পিঠে চড়ে আছে আর দুইজন ...,NaN,দুইজন লোক দুইটা ঘোড়ার পিঠে চড়ে আছে আর দুইজন লো...
8244,8244,8739.png,['সাদা টিশার্ট পরে একটা বাচ্চা ছেলে মুখে হাত দ...,NaN,সাদা টিশার্ট পরে একটা বাচ্চা ছেলে মুখে হাত দিয়...
8245,8245,8740.png,['দুইটা ছোট ছেলে সমুদ্রের পানিতে একটা টায়ারের ...,NaN,দুইটা ছোট ছেলে সমুদ্রের পানিতে একটা টায়ারের উপ...
8246,8246,8741.png,['দুইজন শ্রমিক মাথায় বালুর ঝুড়ি নিয়ে সেই বালু ...,NaN,দুইজন শ্রমিক মাথায় বালুর ঝুড়ি নিয়ে সেই বালু ফে...


In [48]:
print("total input tokens: ", total_input_tokens)
print("total output tokens: ", total_output_tokens)
print("total token count: " , total_tokens)
print(f"total cost: {total_input_tokens * 1e-5 + total_output_tokens * 3e-5}")

total input tokens:  6374566
total output tokens:  958028
total token count:  2758298
total cost: 92.4865


In [49]:
df.isna().sum()

Unnamed: 0         0
filename           0
caption            0
Generated          9
longest_caption    0
dtype: int64

In [36]:
len(df)

8669

In [50]:
df.to_csv("dataset/chitron_concat.csv", index = False)

## Resizing the data 

In [27]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


images = os.listdir("dataset/bnature")
len(images)

8000

In [44]:
for i in images:
    image = cv2.imread(f"dataset/bnature/{i}")
    w, h, c = image.shape
        
    if w > 500 or h > 500:
        try:
            print(image.shape)
            resized = cv2.resize(image, (500, 500))
            print(resized.shape)
            
        except Exception as e:
            print(i, e)
        # cv2.imwrite(f"dataset/bnature/{i}", resized)

    # break

## Concat both csvs

In [7]:
main_df = pd.read_csv("dataset/generated_chitron.csv")
second_df = pd.read_csv("dataset/generated_chitron2.csv")

In [8]:
# Set index to 'filename' for both DataFrames
# main_df.set_index('filename', inplace=True)
# second_df.set_index('filename', inplace=True)

dummy = main_df.combine_first(second_df)


In [9]:
# main_df.head()
input_string = dummy.loc[4016,'Generated']

In [11]:
# dummy.to_csv("dataset/chitron_concat.csv")
# len(dummy)
# 8669 - (2800+517)
dummy = pd.read_csv("dataset/chitron_concat.csv")

In [12]:
dummy.isna().sum()
# dummy.dropna(inplace=True)

Unnamed: 0            0
filename              0
caption               0
Generated          5352
longest_caption       0
dtype: int64

In [227]:
# dummy.head()

In [232]:
def split_qa_string(qa_string):
    # print(qa_string)
    parts = qa_string.split('A#')
    question = parts[0].replace('Q# ', '').strip().strip('",')
    answer = parts[1].strip().strip('"')
    print(question, answer)
    return [question, answer]


In [233]:
dummy['Generated_list'] = dummy['Generated'].apply(split_qa_string)

ছবিতে মহিলাটির হাতে কি আছে? বেত
নৌকায় দাঁড়ানো পুরুষটি কি পরে আছেন? সাদা লুঙ্গী
ল্যাপটপ ধারণকারী মহিলার জামার রঙ কি? নীল
ছবিতে বসে থাকা ছোট ছেলেটি কী ধরনের পোশাক পরে আছে? নীল শার্ট
এই ছবিতে কতজন পুরুষ লুঙ্গী পরে আছেন? একজন
ছবিতে মহিলার পরনে কী ধরনের পোশাক দেখা যাচ্ছে? শাড়ি
পুরুষটি কী ধরণের পোশাক পরে আছেন? কালো পাঞ্জাবী পায়জামা
চাতালে কতজন মানুষ কাজ করছে? ৩ জন
মেয়েটি কি কাজ করছে? ইট ভাঙছে
মহিলাটির কপালে কোন রঙের টিপ রয়েছে? লাল
ছবিতে ছেলেটি কী করছে? পেয়াজ ভরছে
ছবিতে পুরুষটি কী চালাচ্ছে? ব্যান
ছবির মানুষটি কোথায় বসে আছে? কাদাযুক্ত জায়গায়
পাল্লা দিয়ে মাপ নিচ্ছে ব্যক্তির পরনে কী রংয়ের জামা আছে? লাল
মহিলাটি কি ঝাড়ু দিয়ে কীসের নিচে পরিষ্কার করছেন? চালার নিচে
ছবির ছেলেটি কোন পরিবেশে দাঁড়িয়ে আছে? আলো অন্ধকারের মাঝে
ছবিতে বাচ্চা ছেলেটির মাথায় কি পেঁচানো রয়েছে? গামছা
মেয়েদের হাতে কি আছে? একটি বই
ছবিতে কয়জন ছেলে নৌকা থেকে লাফ দিচ্ছে? ২ জন
ছবিতে রাস্তায় কতজন মহিলা ঘুমিয়ে আছেন? একজন
মহিলাটি কোন রঙের জামা উরনা পরে আছেন? আকাশি রঙ
এই ছবিটির ব্যাকগ্রাউন্ডে কোন দেশের পতাকা আঁকা আছে? বাংলাদেশ
ছবিতে কতগুল

In [163]:
parts = input_string.split(', A#')
parts

['Q# "ছবিতে দেখা ফুলটির রঙ কি?"', ' "লাল"']

In [165]:
question = parts[0].replace('Q# ', '').strip().strip('"')
question

'ছবিতে দেখা ফুলটির রঙ কি?'

In [167]:
answer = parts[1].strip().strip('"')
answer

'লাল'

In [168]:
split_qa_string(input_string)

['Q# "ছবিতে দেখা ফুলটির রঙ কি?"', ' "লাল"']


['ছবিতে দেখা ফুলটির রঙ কি?', 'লাল']

## Bnature Data QnA

In [6]:
total_input_tokens = 0#7459341#4842322 #4266080 #3782210 #3677753 #3562644 #2848053 #2288098 #2176137 #405867 # 293207
total_output_tokens = 0#1058581 #785276 #718417 #661913 #649949 #636898 #556623 #493624 # 482592 # 1537880 # 293207
total_tokens = 0#2520663 #2397670 #1393072 # 325355
#  36.23913 # 3.8965100000000006 $  #45.17922
val = 0

In [34]:
df = pd.read_csv("dataset/bnature_generated.csv")
# df.loc[:,'Generated'] = None
df.head()

,image_name,captions,top_captions,Generated
0,1.jpg,"['গ্রামে হাঁটা দুই শিশু।', 'দুই শিশু গ্রামে হা...","দুই শিশু গ্রামে হাঁটছে।,\n দুই শিশু খেলা করে।,...",Q# ছবিতে দেখা মেয়েটি কি করছে? \nA# দাঁড়িয়ে আছে
1,2.jpg,"['কিছু লোক নদী পার করছে।', 'কিছু লোক বিল পার ক...","অনেকগুলো লোক নদী পার করছে।,\n কিছু লোক নদী পার...",Q# ছবিতে দেখা লোকগুলি কোন পরিবেশ পার করছে? \n...
2,3.jpg,"['এক কৃষক ক্ষেতে কাজ করছে।', 'এক কৃষক ধান ক্ষে...","রোদ এর নিচে একটা লোক দাঁড়িয়ে আছে।,\n এক কৃষক...",Q# ছবিতে দেখা লোকটি কি করছে?\nA# ধান দেখছে
3,4.jpg,"['কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।', '...","কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।,\n সব...",Q# ছবিতে দেখা যাচ্ছে শিশুরা কোথা থেকে আসছে? \...
4,5.jpg,"['পাঁচটি নারী এক জায়গায় চলছে।', 'পাঁচটি শিশু...","পাঁচটি নারী এক জায়গায় চলছে।,\n পাঁচটি মেয়ে ...",Q# ছবিতে কতগুলি মেয়ে চলছে?\nA# পাঁচটি


In [35]:
import time

print(len(df))
print(df.isna().sum())

8000
image_name         0
captions           0
top_captions       0
Generated       5600
dtype: int64


In [36]:
sliced_df = df[2285:2400]
sliced_df.isna().sum()

image_name      0
captions        0
top_captions    0
Generated       0
dtype: int64

In [29]:
for idx, rows in sliced_df.iterrows():
    res = (give_q_n_a(rows['image_name'], rows['top_captions']))

    total_input_tokens = total_input_tokens + res['usage']['prompt_tokens']
    total_output_tokens = total_output_tokens + res['usage']['completion_tokens']
    total_tokens = total_tokens + res['usage']['total_tokens']
    
    df.loc[df['image_name'] == str(rows['image_name']), 'Generated'] = str(res['choices'][0]['message']['content'])
    
    print("done: idx: ",idx," image_name: ", str(rows['image_name']), "input_token: ", res['usage']['prompt_tokens'], "output_token: ", res['usage']['completion_tokens'])
    val = val+1
    time.sleep(1)
    # break

done: idx:  2285  image_name:  2286.jpg input_token:  588 output_token:  76
done: idx:  2286  image_name:  2287.jpg input_token:  585 output_token:  58
done: idx:  2287  image_name:  2288.jpg input_token:  544 output_token:  43
done: idx:  2288  image_name:  2289.jpg input_token:  585 output_token:  83
done: idx:  2289  image_name:  2290.jpg input_token:  556 output_token:  71
done: idx:  2290  image_name:  2291.jpg input_token:  573 output_token:  71
done: idx:  2291  image_name:  2292.jpg input_token:  557 output_token:  53
done: idx:  2292  image_name:  2293.jpg input_token:  574 output_token:  29
done: idx:  2293  image_name:  2294.jpg input_token:  560 output_token:  41
done: idx:  2294  image_name:  2295.jpg input_token:  554 output_token:  49
done: idx:  2295  image_name:  2296.jpg input_token:  543 output_token:  42
done: idx:  2296  image_name:  2297.jpg input_token:  608 output_token:  69
done: idx:  2297  image_name:  2298.jpg input_token:  582 output_token:  46
done: idx:  

In [13]:
res

{'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.',
  'type': 'insufficient_quota',
  'param': None,
  'code': 'insufficient_quota'}}

In [14]:
sliced_df.head(10)

,image_name,captions,top_captions,Generated
2104,2105.jpg,"['একটি নৌকায় দুজনকে দেখা যাচ্ছে।', 'ডিঙি নৌকাত...",একটি ছোট নৌকায় ২ জন মানুষ এবং পাশে একটি বড় ন...,"Q# ""ছবিতে কত জন মানুষ ডিঙি নৌকায় রয়েছেন?"", A..."
2105,2106.jpg,"['একটি শিশু আছে।', 'সাদা জামা পড়া একটি শিশু আছ...",সবুজ গ্রাসের উপরে সাদা জামা পরে দাঁড়িয়ে আছেন...,NaN
2106,2107.jpg,"['একটি শিশু হাতে ফুল নিয়ে আছে।', 'একটা বাচ্চা ...",একজন ছোট মেয়ে গোলাপি জামা পরে হাতে কচুরিপানার...,NaN
2107,2108.jpg,"['একজন মেয়ে শিশু আছে। পিছনে একটি গরু আছে।', 'এ...",জমির উপর দিয়ে একটি গরু নিয়ে আসছেন একজন ছোট ম...,NaN
2108,2109.jpg,"['জলাশয়ের পারে একজন মানুষ বসে আছে।', 'গাছের নি...","ঘুড়ি উরাবার জন্য বসেছে, নদীতে ঘুড়ি দেখা যাচ্...",NaN
2109,2110.jpg,"['তিনজন মানুষ আছে।', 'কয়েকজনকে কাজ করতে দেখা য...",জমির উপর এবং পানিতে পাঁটের গাছের বেলা নিয়ে কা...,NaN
2110,2111.jpg,['একজন পুরুষ পাতা বোঝাই করে ভ্যান চালিয়ে আসছে।...,চালকের কষ্ট হচ্ছে তাও একজন ভ্যানচালক তা টেনে ন...,NaN
2111,2112.jpg,"['একজন পুরুষ আছে।', 'একজনকে দেখা যাচ্ছে।', 'এক...",একজন পুরুষ একটি বাড়ীর উঠানে গাছের নিচে দাঁড়ি...,NaN
2112,2113.jpg,"['একজন বালক আছে।', 'একটি শিশু দাঁড়িয়ে আছে।', '...","হাসি মাখা মুখ নিয়ে একটি শিশু দাঁড়িয়ে আছে।,\...",NaN
2113,2114.jpg,"['রাস্তার পাশে একজন পুরুষ বসে আছে।', 'একজন বৃদ...",একজন বৃদ্ধ লোক বসে আছে তার পেছন অনেক গাড়ীর টা...,NaN


In [30]:
print("total input tokens: ", total_input_tokens)
print("total output tokens: ", total_output_tokens)
print("total token count: " , total_tokens)
print(f"total cost: {total_input_tokens * 1e-5 + total_output_tokens * 3e-5}")

total input tokens:  171611
total output tokens:  17347
total token count:  188958
total cost: 2.2365200000000005


In [24]:
df.head()

,image_name,captions,top_captions,Generated
0,1.jpg,"['গ্রামে হাঁটা দুই শিশু।', 'দুই শিশু গ্রামে হা...","দুই শিশু গ্রামে হাঁটছে।,\n দুই শিশু খেলা করে।,...",Q# ছবিতে দেখা মেয়েটি কি করছে? \nA# দাঁড়িয়ে আছে
1,2.jpg,"['কিছু লোক নদী পার করছে।', 'কিছু লোক বিল পার ক...","অনেকগুলো লোক নদী পার করছে।,\n কিছু লোক নদী পার...",Q# ছবিতে দেখা লোকগুলি কোন পরিবেশ পার করছে? \n...
2,3.jpg,"['এক কৃষক ক্ষেতে কাজ করছে।', 'এক কৃষক ধান ক্ষে...","রোদ এর নিচে একটা লোক দাঁড়িয়ে আছে।,\n এক কৃষক...",Q# ছবিতে দেখা লোকটি কি করছে?\nA# ধান দেখছে
3,4.jpg,"['কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।', '...","কিছু শিশু তাদের স্কুল থেকে বাড়িতে যায়।,\n সব...",Q# ছবিতে দেখা যাচ্ছে শিশুরা কোথা থেকে আসছে? \...
4,5.jpg,"['পাঁচটি নারী এক জায়গায় চলছে।', 'পাঁচটি শিশু...","পাঁচটি নারী এক জায়গায় চলছে।,\n পাঁচটি মেয়ে ...",Q# ছবিতে কতগুলি মেয়ে চলছে?\nA# পাঁচটি


In [31]:
df.to_csv("dataset/bnature_generated.csv", index=False)